In [1]:
#!pip install -U spacy --user
#!pip install -U https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.0.0/en_core_web_sm-2.0.0.tar.gz --user
#!python -m spacy download en_core_web_sm

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import en_core_web_sm

In [3]:
nlp = en_core_web_sm.load()

In [4]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"] 

In [5]:
import json
json_data = open("./anly540_apple.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))

In [6]:
json_data[2]

'{"thread": {"uuid": "2587a9af973a7900eef166282577cea4e7f32b65", "url": "http://omgili.com/ri/.wHSUbtEfZQTKtctguhP2.h4YK4R6rP141R4tpsKCfOicasVzd5vyEtWnTr19ghA8zw8ETsgsbUTihYjrtq.A6.af1dMhyuDM_HMLhb4vmptEAxXW5omzAGS7WLS9MO_4tzIfBZ9Khk-", "site_full": "wuzzuf.net", "site": "wuzzuf.net", "site_section": "http://wuzzuf.net/feeds/all-jobs.xml", "site_categories": ["job_search", "jobs"], "section_title": "Wuzzuf All Jobs", "title": "Web Developer Intern - Bahrain", "title_full": "Web Developer Intern - Bahrain", "published": "2018-10-04T03:40:00.000+03:00", "replies_count": 0, "participants_count": 0, "site_type": "blogs", "country": "IE", "spam_score": 0.0, "main_image": "https://s3-eu-west-1.amazonaws.com/wuzzuf/files/company_logo/AIESEC--MANSOURA-Egypt-31084-1522674809-og.png", "performance_score": 0, "domain_rank": 15090, "social": {"facebook": {"likes": 0, "comments": 0, "shares": 0}, "gplus": {"shares": 0}, "pinterest": {"shares": 0}, "linkedin": {"shares": 0}, "stumbledupon": {"shares

In [7]:
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))

In [8]:
for feed in feeds_read_from_file[3:4]:
    selected_feed = feed['text']
    print(selected_feed)            

Nintendo is planning an update to the Switch, according to new reports.The new console is likely to arrive next year, the Wall Street Journal said. But beyond that details remain murky.The Switch was first released in March of last year. The new update is being planned for the second half 2019 and could come as soon as the summer, according to the WSJ.It isn't clear what updates the new version of the console would bring. But chief among them is likely to be a new screen: the first Switch, while broadly praised, drew some criticism for the quality of the display.Nintendo is said to be avoiding the OLED technology that is used in Apple's new iPhones and which can give improved performance and rich blacks. But it could improve to the more detailed LCD screens on other recent handsets.That uncertainly might partly be because Nintendo itself isn't exactly clear what the new console might look like.If it does arrive, it's likely that it will follow the lead of recent new consoles like the P

In [9]:
import re
from nltk import word_tokenize, sent_tokenize, ngrams
sentences = sent_tokenize(selected_feed)
for sentence in sentences:
    print(sentence)

Nintendo is planning an update to the Switch, according to new reports.The new console is likely to arrive next year, the Wall Street Journal said.
But beyond that details remain murky.The Switch was first released in March of last year.
The new update is being planned for the second half 2019 and could come as soon as the summer, according to the WSJ.It isn't clear what updates the new version of the console would bring.
But chief among them is likely to be a new screen: the first Switch, while broadly praised, drew some criticism for the quality of the display.Nintendo is said to be avoiding the OLED technology that is used in Apple's new iPhones and which can give improved performance and rich blacks.
But it could improve to the more detailed LCD screens on other recent handsets.That uncertainly might partly be because Nintendo itself isn't exactly clear what the new console might look like.If it does arrive, it's likely that it will follow the lead of recent new consoles like the P

In [10]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [11]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [12]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [13]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [14]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [15]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [16]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [17]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [18]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [19]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [20]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [21]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [22]:
def getAbuserOntoVictimSVOs(tokens):
    maleAbuser = {'he', 'boyfriend', 'bf', 'father', 'dad', 'husband', 'brother', 'man'}
    femaleAbuser = {'she', 'girlfriend', 'gf', 'mother', 'mom', 'wife', 'sister', 'woman'}
    neutralAbuser = {'pastor', 'abuser', 'offender', 'ex', 'x', 'lover', 'church', 'they'}
    victim = {'me', 'sister', 'brother', 'child', 'kid', 'baby', 'friend', 'her', 'him', 'man', 'woman'}

    svos = findSVOs(tokens)
    wnl = WordNetLemmatizer()
    passed = []
    for s, v, o in svos:
        s = wnl.lemmatize(s)
        v = "!" + wnl.lemmatize(v[1:], 'v') if v[0] == "!" else wnl.lemmatize(v, 'v')
        o = "!" + wnl.lemmatize(o[1:]) if o[0] == "!" else wnl.lemmatize(o)
        if s in maleAbuser.union(femaleAbuser).union(neutralAbuser) and o in victim:
            passed.append((s, v, o))
    return passed

In [23]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs():
    #nlp = English()
    for sentence in sentences:
        tok = nlp(sentence)
        svos = findSVOs(tok)
    #     printDeps(tok)
    #    assert set(svos) == {('i', '!have', 'assistance'), ('he', '!provide', 'support')}
        if svos:
            for svo in svos:
                print('S:{0}, V:{1}, O:{2}'.format((svo[0]), svo[1], svo[2]))
        else:
            print('No SVO')

In [24]:
if __name__ == "__main__":
    testSVOs()

S:nintendo, V:planning, O:update
No SVO
S:what, V:updates, O:version
S:nintendo, V:avoiding, O:technology
S:which, V:give, O:performance
S:which, V:give, O:blacks
S:it, V:follow, O:lead
S:players, V:bring, O:games
S:players, V:bring, O:friends
S:nintendo, V:done, O:that
